In [ ]:
import json
from py2neo import Graph, Node, Relationship
#from py2neo.Graph import database 

# Need to get authentication working, currently NEO4J_AUTH=none
graph = Graph("bolt://neo4j:7687")
# graph = Graph('bolt://localhost:7687', bolt=True)

#graph.delete_all()

n_nodes = graph.database.primitive_counts['NumberOfNodeIdsInUse']
n_relationships = graph.database.primitive_counts['NumberOfRelationshipIdsInUse']
print("Connected to graph database with {:,} nodes and {:,} relationships!".format
     (n_nodes, n_relationships))

In [ ]:
##import the publications where lang = 'en' and publisher = "Science" or "Nature" in year 2008
import pandas as pd
import time
print("load english science and nature publication into df")
start_time = time.time()
query = """
MATCH (n:Quanta) WHERE n.lang = 'en' AND ( n.venue = 'Science' OR n.venue = 'Nature') AND n.year =2008 AND EXISTS(n.fos)
RETURN 
n.venue as venue,
n.pageRank_2018 as PR_2018,
n.pageRank_2008 as PR_2008,
n.fos as fos,
n.title as title,
n.keywords as keywords,
n.publisher as publisher
ORDER BY n.pageRank_2018 DESC
"""
#n.keywords as keywords
dfs_2008_test = graph.run(query).to_data_frame()
end_time = time.time()
print("Finished all calculations in {:.2f} minutes.".format((end_time-start_time)/60))
#dfs_2008_test

## 1.2 Define publiction is popular in 2018 if log(PR_2018) > -0.5 and append the result into dataframe

In [ ]:
import math
start_time = time.time()
count = len(dfs_2008_test["PR_2018"].tolist())
pr_2018_list = dfs_2008_test["PR_2018"].tolist()
popular_result = []
for pr in pr_2018_list:
    if math.log(pr,10) > -0.5:
        popular_result.append(1)
    else:
        popular_result.append(0)

print(len(popular_result))

dfs_2008_test['popular'] = pd.Series(popular_result).values
end_time = time.time()
print("Finished all calculations in {:.2f} minutes.".format((end_time-start_time)/60))
dfs_2008_test

In [ ]:
#check the distribution of high-impact and the rest
count_0 = 0
count_1 = 0
for i in popular_result:
    if i == 1:
        count_1 += 1
    else:
        count_0 += 1
print(count_0)
print(count_1)
print(count_1/len(popular_result))

# 2. Data Exploration
## 2.1 PR_2008 compared to PR_2018

In [ ]:
#visualize histogram bar chart for PR_2018
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
pr_2018_list = dfs_2008_test["PR_2018"].tolist()
# %matplotlib inline
# plt.hist(pr_2018_list, normed=True, bins=30)
# plt.ylabel('Probability');
counts, bins, patches = plt.hist(pr_2018_list, bins = 7)
plt.ylabel('Count')

# Label the raw counts and the percentages below the x-axis...
bin_centers = 0.5 * np.diff(bins) + bins[:-1]
for count, x in zip(counts, bin_centers):
    # Label the raw counts
    plt.annotate('{:.2f}'.format(count), xy=(x, 0), xycoords=('data', 'axes fraction'),
        xytext=(0, 18), textcoords='offset points', va='top', ha='center')

plt.show()

In [ ]:
# visualize the bar without the smallest value
pr_2018_smallest = pr_2018_list[-1]
pr_2018_list_wo_smallest = []
count = 0
for pr in pr_2018_list:
    if pr>pr_2018_smallest:
        pr_2018_list_wo_smallest.append(pr)
    else:
        count += 1

counts, bins, patches = plt.hist(pr_2018_list_wo_smallest)
plt.ylabel('Count')

# Label the raw counts and the percentages below the x-axis...
bin_centers = 0.5 * np.diff(bins) + bins[:-1]
for count, x in zip(counts, bin_centers):
    # Label the raw counts
    plt.annotate('{:.2f}'.format(count), xy=(x, 0), xycoords=('data', 'axes fraction'),
        xytext=(0, 18), textcoords='offset points', va='top', ha='center')

plt.show()
print(count)

In [ ]:
#visualiaze the PageRank distribution after log
pr_2018_list_log = []
for pr in pr_2018_list:
    pr_2018_list_log.append(math.log(pr,10))

# %matplotlib inline
# plt.hist(pr_2018_list, normed=True, bins=30)
# plt.ylabel('Probability');
counts, bins, patches = plt.hist(pr_2018_list_log,bins = 8)
plt.ylabel('Count')

# Label the raw counts and the percentages below the x-axis...
bin_centers = 0.5 * np.diff(bins) + bins[:-1]
for count, x in zip(counts, bin_centers):
    # Label the raw counts
    plt.annotate('{:.2f}'.format(count), xy=(x, 0), xycoords=('data', 'axes fraction'),
        xytext=(0, 18), textcoords='offset points', va='top', ha='center')

plt.show()

# 3. Feature Engneering
## 3.1 one-hot encoding in fos

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
import itertools

dfs_2008_test_copy = dfs_2008_test.copy()
start_time = time.time()
fos_list = dfs_2008_test_copy["fos"].tolist()
fos_list = [[] if v is None else v for v in fos_list]


dfs_2008_test_copy.head()
#Replace original fos with updated fos 
dfs_2008_test_copy['fos'] = pd.Series(fos_list).values
dfs_2008_test

mlb = MultiLabelBinarizer()
X = mlb.fit_transform(dfs_2008_test_copy.fos)
dfs_2008_test_copy = dfs_2008_test_copy.join(pd.DataFrame(X, columns=mlb.classes_))

#del fos in the df

del dfs_2008_test_copy['fos']
end_time = time.time()
print("Finished all calculations in {:.2f} minutes.".format((end_time-start_time)/60))
dfs_2008_test_copy

## 3.2 bag of words model for titles


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(dfs_2008_test.title).toarray()
#labels = df.category_id
print(features.shape)

print(len(tfidf.get_feature_names()))
print(type(features))

title_feature_name = tfidf.get_feature_names()
count_featurename = len(title_feature_name)
for i in range(count_featurename):
    column_name = 'title_'+title_feature_name[i]
    dfs_2008_test_copy[column_name] = pd.Series(features[:,i]).values

dfs_2008_test_copy

In [ ]:

del dfs_2008_test_copy['title']
dfs_2008_test_copy.head()

## 3.3 one-hot encoding for venues

In [ ]:
#data exploration
venue_list = dfs_2008_test_copy["venue"].tolist()
popular_list = dfs_2008_test_copy["popular"].tolist()

print(dfs_2008_test_copy["venue"].value_counts()[:20])
list_len = len(popular_list)
nature_count_1 = 0
nature_count_0 = 0
science_count_1  = 0 
science_count_0 = 0 

for i in range(list_len):
    if venue_list[i] == 'Science' and popular_list[i]==1:
        science_count_1 +=1
    elif venue_list[i] == 'Science' and popular_list[i]==0:
        science_count_0 +=1
    elif venue_list[i] == 'Nature' and popular_list[i]==1:
        nature_count_1 +=1
    else:
        nature_count_0 +=1
        
        
print('science_count_1: ' + str(science_count_1))
print('science_count_0: ' + str(science_count_0))
print('nature_count_1: ' + str(nature_count_1))
print('nature_count_0: ' + str(nature_count_0))
print('science_1 percentage: ' + str(science_count_1/(science_count_1+science_count_0)))
print('nature_1 percentage: ' + str(nature_count_1/(nature_count_1+nature_count_0)))

In [ ]:
dfDummies = pd.get_dummies(dfs_2008_test_copy['venue'], prefix = 'venue')
dfs_2008_test_copy = pd.concat([dfs_2008_test_copy, dfDummies], axis=1)
del dfs_2008_test_copy['venue']
del dfs_2008_test_copy['keywords']
del dfs_2008_test_copy['publisher']
del dfs_2008_test_copy['PR_2008']
dfs_2008_test_copy

## 3.4 2008_2018 conversion

In [ ]:
#2008 ranking compare to 2018 ranking 

title_list = dfs_2008_test["title"].tolist()
PR_2008_score_list = dfs_2008_test["PR_2008"].tolist()
PR_2018_score_list = dfs_2008_test["PR_2018"].tolist()

# create the ranking list of 2018
sort_unique_2018 = sorted(set(PR_2018_score_list))
dic_2018_rank = {}
count = 0
for i in sort_unique_2018:
    dic_2018_rank[i] = len(sort_unique_2018)-count
    count += 1
PR_2018_rank_list = [] 
for i in PR_2018_score_list:
    PR_2018_rank_list.append(dic_2018_rank[i])

# create the ranking list of 2008
sort_unique_2008 = sorted(set(PR_2008_score_list))
dic_2008_rank = {}
count = 0
for i in sort_unique_2008:
    dic_2008_rank[i] = len(sort_unique_2008)-count
    count += 1
PR_2008_rank_list = [] 
for i in PR_2008_score_list:
    PR_2008_rank_list.append(dic_2008_rank[i])

In [ ]:
## visualization 2 list

id_2018 = []
for i in range(len(PR_2008_score_list)):
    id_2018.append(i)

plt.plot(id_2018, PR_2018_rank_list, color='g')
plt.plot(id_2018, PR_2008_rank_list, color='orange')
plt.xlabel('id')
plt.ylabel('PageRank Impact Ranking')
plt.show()

In [ ]:
id_2018 = []
for i in range(len(PR_2008_score_list)):
    id_2018.append(i)

plt.plot(id_2018[:200], PR_2018_rank_list[:200], color='g')
plt.plot(id_2018[:200], PR_2008_rank_list[:200], color='orange')
plt.xlabel('id')
plt.ylabel('PageRank Impact Ranking')
plt.show()

# 4.  Model Construction
## 4.1 Random forest
### 4.1.1 random forest with featured venue, title and fos

In [ ]:
popular_2018 = dfs_2008_test["popular"].tolist()
del dfs_2008_test_copy['popular']
del dfs_2008_test_copy['PR_2018']
dfs_2008_test_copy['popular'] = pd.Series(popular_2018).values
dfs_2008_test_copy.head()

In [ ]:
# get the comlumn list
from sklearn.cross_validation import train_test_split
feature_list = list(dfs_2008_test_copy.columns.values)
# split train and test
df_X = dfs_2008_test_copy[feature_list[:-1]]
df_y = dfs_2008_test_copy['popular']
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.3, random_state=99)

In [ ]:
#random forest classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print('accuracy: ', metrics.accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
conf = confusion_matrix(y_test, y_pred)
print(conf)
tp = conf[0][0]
fp = conf[0][1]
fn = conf[1][0]
tn = conf[1][1]

print('sensitivity: ')
print(tp/(tp+fn))
print('sprcificity: ')
print(tn/(fp+tn))

## 2.1.2 Add 2008 pagerank score into features 

In [ ]:
del dfs_2008_test_copy['popular']
dfs_2008_test_copy['PR_2008'] = pd.Series(PR_2008_score_list).values
dfs_2008_test_copy['popular'] = pd.Series(popular_2018).values

In [ ]:
from sklearn.cross_validation import train_test_split
feature_list = list(dfs_2008_test_copy.columns.values)
# split train and test
df_X = dfs_2008_test_copy[feature_list[:-1]]
df_y = dfs_2008_test_copy['popular']
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.3, random_state=99)

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print('accuracy: ', metrics.accuracy_score(y_test, y_pred))

from sklearn.metrics import confusion_matrix
conf = confusion_matrix(y_test, y_pred)
print(conf)
tp = conf[0][0]
fp = conf[0][1]
fn = conf[1][0]
tn = conf[1][1]

print('sensitivity: ')
print(tp/(tp+fn))
print('specificity: ')
print(tn/(fp+tn))

In [ ]:
## comment: we could see that the PR_score in the pulished year could improve the accuracy for the next ten years
## However, the problem here is we for some publications published in Dec, it might not have rnough time to acculumate 
## the PR score.

#cross validation
from sklearn import tree
from sklearn import cross_validation
from sklearn.grid_search import GridSearchCV
start_time = time.time()

param_grid = {'min_samples_split':[5,6,7],
             'n_estimators': [80,100,120]}

clf = RandomForestClassifier()
grid_search = GridSearchCV(estimator = clf, param_grid = param_grid, cv=3, n_jobs =1, refit = False )
grid_search.fit(X = df_X, y = df_y)

print(grid_search.best_params_, grid_search.best_score_)

end_time = time.time()
print("Finished all calculations in {:.2f} minutes.".format((end_time-start_time)/60))

# plays fine in cross validation

## 4.2 Logistic regression
### 4.2.1 with fos venue and title 

In [ ]:
del dfs_2008_test_copy['PR_2008']
dfs_2008_test_copy.head()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
feature_list = list(dfs_2008_test_copy.columns.values)
# split train and test
df_X = dfs_2008_test_copy[feature_list[:-1]]
df_y = dfs_2008_test_copy['popular']
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.3, random_state=99)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

from sklearn.metrics import confusion_matrix
conf = confusion_matrix(y_test, y_pred)
print(conf)
tp = conf[0][0]
fp = conf[0][1]
fn = conf[1][0]
tn = conf[1][1]

print('sensitivity: ')
print(tp/(tp+fn))
print('sprcificity: ')
print(tn/(fp+tn))

In [ ]:
del dfs_2008_test_copy['popular']
dfs_2008_test_copy['PR_2008'] = pd.Series(PR_2008_score_list).values
dfs_2008_test_copy['popular'] = pd.Series(popular_2018).values
dfs_2008_test_copy.head()

In [ ]:
feature_list = list(dfs_2008_test_copy.columns.values)
# split train and test
df_X = dfs_2008_test_copy[feature_list[:-1]]
df_y = dfs_2008_test_copy['popular']
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.3, random_state=99)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

from sklearn.metrics import confusion_matrix
conf = confusion_matrix(y_test, y_pred)
print(conf)
tp = conf[0][0]
fp = conf[0][1]
fn = conf[1][0]
tn = conf[1][1]

print('sensitivity: ')
print(tp/(tp+fn))
print('sprcificity: ')
print(tn/(fp+tn))

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

# 5.1 Deep learning model

In [ ]:
value_array = dfs_2008_test_copy.values
print(value_array.shape)
y = value_array[:, -1]
# print(y)
X = value_array[:, :-1]
print(X.shape)
print(len(X[1]))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense
#Initializing Neural Network
classifier = Sequential()
# Adding the input layer and the first hidden layer
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu', input_dim = len(X[1])))
# Adding the second hidden layer
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu'))
# Adding the output layer
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))

In [ ]:
# Compiling Neural Network
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
# Fitting our model 
classifier.fit(X_train, y_train, batch_size = 10, nb_epoch = 100)


In [ ]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)
# Creating the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
print(cm)
tp = cm[0][0]
fp = cm[0][1]
fn = cm[1][0]
tn = cm[1][1]

print('sensitivity: ')
print(tp/(tp+fn))
print('specificity: ')
print(tn/(fp+tn))

print('accuracy: ')
print((tp+tn)/(tp+tn+fp+fn))

In [ ]:
# roc curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
nn_roc_auc = roc_auc_score(y_test, y_pred)
fpr, tpr, thresholds = roc_curve(y_test, classifier.predict_proba(X_test))
plt.figure()
plt.plot(fpr, tpr, label='NN Regression (area = %0.2f)' % nn_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Neural Network ROC curve')
plt.legend(loc="lower right")
plt.show()